In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
    
engine = create_engine('mysql+pymysql://root@localhost:3306/music_development')
music = engine.connect()
data_path = '../data/'

In [5]:
sql = '''
SELECT * FROM songs
WHERE rank BETWEEN 1130 AND 1140
ORDER BY rank DESC
'''
songs = pd.read_sql(sql, music)
songs

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,629,1137,Landslide,29,211,Fleetwood_Mac_-_Landslide.mp3,1,WM7-PYtXtJM,1940-01-04,0,0
1,628,1136,Gypsy,29,264,Fleetwood_Mac_-_Gypsy.mp3,1,mV_dnrxD6v4,1940-01-04,0,0
2,627,1135,Numb,339,187,Linkin_Park_-_Numb.mp3,1,kXYiU_JCYtU,1940-01-03,0,0
3,626,1134,New Divide,339,268,Linkin_Park_-_New_Divide.mp3,1,ysSxxIqKNN0,1940-01-03,0,0
4,625,1133,One More Light,339,255,Linkin_Park_-_One_More_Light.mp3,2,xC2s4HAWoXA,1940-01-02,0,0
5,624,1132,Cat's In The Cradle,338,223,Harry_Chapin_-_Cat_s_In_The_Cradle.mp3,1,OcNsiccdDyU,1940-01-02,0,0
6,623,1131,Morning Has Broken,231,191,Cat_Stevens_-_Morning_Has_Broken.mp3,1,j_efw1-3OvU,1940-01-02,0,0
7,622,1130,The Moon Represents My Heart (CN),337,203,Teresa_Teng_-_The_Moon_Represents_My_Heart__CN...,1,bv_cEeDlop0,1940-01-01,0,0


In [7]:
songs.dtypes

id                int64
rank              int64
name             object
artist_id         int64
length            int64
location         object
genre_id          int64
youtube_code     object
released_date    object
rating            int64
website           int64
dtype: object

In [11]:
sql = '''
SELECT * FROM artists WHERE id = 80
'''
artists = pd.read_sql(sql, music)
artists

,id,first_name,last_name,band,image
0,80,Rod,Stewart,0,Rod_Stewart.jpg


In [27]:
sql = '''
SELECT * FROM songs
WHERE LOWER(TRIM(name)) LIKE LOWER(TRIM('I Don''t Want to Talk About It'))
'''
songs = pd.read_sql(sql, engine)
songs 

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,612,1120,I Don't Want to Talk About It,80,290,Rod_Stewart_-_I_Don_t_Want_to_Talk_About_It.mp3,1,w46bWxS9IjY,2025-12-29,0,0


In [19]:
sql = '''
SELECT * FROM songs
WHERE id = 612
'''
songs = pd.read_sql(sql, engine)
songs

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,612,1120,I Don't Want to Talk About It,80,290,Rod_Stewart_-_I_Don_t_Want_to_Talk_About_It.mp3,1,w46bWxS9IjY,2025-12-29,0,0


In [29]:
song = songs.iloc[0]
songId = song['id']
songId

612

In [35]:
sql = """SELECT lyrics.* 
FROM lyrics 
WHERE lyrics.song_id IS NOT NULL 
  AND lyrics.song_id NOT IN (SELECT id FROM songs)"""
lyrics = pd.read_sql(sql, engine)
lyrics

,id,song_id,content,created_at,updated_at
0,286,417,"Angie, Angie, when will those clouds all disap...",2018-02-23 13:11:48,2018-02-23 13:11:48


In [31]:
# Method 2: Direct string printing with proper formatting
sql = f"""
SELECT id, song_id, content 
FROM lyrics 
WHERE song_id = {songId}
"""
lyric = pd.read_sql(sql, engine)
lyric

,id,song_id,content
0,538,612,10|[Music]\r\n50|♪♪♪\r\n100|[Music]\r\n150|♪♪♪...


In [7]:
lyric.dtypes

id          int64
song_id     int64
content    object
dtype: object

In [8]:
sql = text("""
SELECT artist_id, A.first_name, A.last_name, COUNT(*) AS nbr
FROM songs S 
JOIN artists A
ON S.artist_id = A.id
GROUP BY artist_id
HAVING COUNT(*) > 4
ORDER BY COUNT(*) DESC
""")
df = pd.read_sql(sql, engine)
df

,artist_id,first_name,last_name,nbr
0,1,Beatles,The,26
1,3,Rolling Stones,The,16
2,110,Bee Gees,,11
3,264,Taylor,Swift,11
4,82,Elton,John,10
5,2,Bob,Dylan,10
6,15,Elvis,Presley,9
7,38,Stevie,Wonder,7
8,25,Creedence Clearwater Revival,,7
9,31,Led Zeppelin,,6


In [9]:
sql = text("""
SELECT artist_id, A.first_name, A.last_name, S.name, S.rank, S.id
FROM songs S 
JOIN artists A
ON S.artist_id = A.id
WHERE first_name = 'CEline'
ORDER BY rank
""")
df = pd.read_sql(sql, engine)
df

,artist_id,first_name,last_name,name,rank,id
0,174,Celine,Dion,Because You Loved Me,563,294
1,174,Celine,Dion,My Heart Will Go On,564,293
2,174,Celine,Dion,I'm Alive,565,292
3,174,Celine,Dion,All By Myself,566,291
4,174,Celine,Dion,The Power of Love,567,290
